# JOIN

Join is a keyword that allows you to combine information from different tables. It's really important, but sometimes it's hard to understand, so this whole page is about features and usecases of joins in Postgres SQL.

In the following cell, I'm creating a database that will be used for all the examples on this page.

In [1]:
docker run -d --rm\
    --name join_example_postgres\
    -e POSTGRES_PASSWORD=postgres\
    postgres:15.4 &> /dev/null
sleep 5

**Note** Remember to close the container when you have finished playing with the examples.

In [12]:
docker stop join_example_postgres

join_example_postgres


## Join types

This diagram describes the different types of joins available in SQL.

![join types](join_files/join_types.png)

## Short names

You can use short names for the tables involved in the join to make it easier to write code. Just give the new name of the table separated by a space from the original name of the table.

---

The following cell shows how tables `table1` and `table2`, aliased as `t1` and `t2` respectively, are joined and used in a query.

In [7]:
docker exec -i join_example_postgres psql -U postgres -d postgres << EOF
WITH

table1(col1, col2) AS (
    VALUES
    (1, 'a'),
    (2, 'b'),
    (3, 'c'),
    (4, 'd')
),
table2(col1, col2) AS (
    VALUES
    (1, 'A'),
    (2, 'B'),
    (3, 'C'),
    (4, 'D')
)

SELECT * 
FROM 
    table1 t1 JOIN table2 t2 
    ON (t1.col1=t2.col1);
EOF

 col1 | col2 | col1 | col2 
------+------+------+------
    1 | a    |    1 | A
    2 | b    |    2 | B
    3 | c    |    3 | C
    4 | d    |    4 | D
(4 rows)



## Several tables

You can use more than two tables in a join. Just use multiple `JOIN ... ON ...` blocks to select more information.

---

In the following example, all information is combined into a single query using multiple joins.

In [17]:
docker exec -i join_example_postgres psql -U postgres -d postgres << EOF

WITH 
table1(col1) AS (VALUES (1), (2), (3)),
table2(col1) AS (VALUES (1), (3), (2)),
table3(col1) AS (VALUES (3), (1), (7))

SELECT *
FROM 
    table1
    JOIN table2 ON table1.col1=table2.col1
    JOIN table3 ON table1.col1=table3.col1
;
EOF

 col1 | col1 | col1 
------+------+------
    1 |    1 |    1
    3 |    3 |    3
(2 rows)

